In [ ]:
function Franke(x,y)
    p1 = 3/4*ℯ^(-(9x-2)^2/4-(9y-2)^2/4)
    p2 = 3/4*ℯ^(-(9x+1)^2/49-(9y+1)^2/10)
    p3 = 1/2*ℯ^(-(9x-7)^2/4-(9y-3)^2/4)
    p4 = -1/5*ℯ^(-(9x-4)^2-(9y-7)^2)
    return p1+p2+p3+p4
end
"""
    SplitData2D(xData, yData, zData, train_size)

Splits data with given split: train_size


# Input
 - xData::Array:        Data in X  
 - yData::Array:        Data in Y
 - zData::Array:        Data in Z
 - train_size::Float64: Proportion of data to be given as trainingdata

"""
function SplitData2D(xData::Array, yData::Array, zData::Array, train_size::Float64)
    # Assumes xData and yData are vectors, and zData is a matrix representing the grid

    # Flatten the Z matrix and corresponding X, Y arrays
    xGrid = repeat(xData, inner=length(yData))
    yGrid = repeat(yData, outer=length(xData))
    zVector = vec(zData)

    # Number of total samples
    N = length(zVector)
    
    # Determine number of training samples
    NumTrain = Int(round(N * train_size))
    
    # Randomly sample indices for training data
    train_indices = sample(1:N, NumTrain; replace=false)
    
    # Determine the test indices
    all_indices = collect(1:N)
    test_indices = [i for i in all_indices if i ∉ train_indices]
    
    # Create training and testing datasets
    xTrain = xGrid[train_indices]
    yTrain = yGrid[train_indices]
    zTrain = zVector[train_indices]
    
    xTest = xGrid[test_indices]
    yTest = yGrid[test_indices]
    zTest = zVector[test_indices]
    
    return xTrain, yTrain, zTrain, xTest, yTest, zTest
end
"""
    DataProjector(x, y, z, numDegrees;train, normalize)

This function Projects the Data from the type we generate to the type we need for Regression.


# Input
 - x:        Data in X  
 - y:        Data in Y
 - z:        Data in Z
 - numDegrees: The number of orders of the polynomial
 - train: Whether or not the data should be handeled as training data
 - normalize: Whether or not the DesignMatrix should be normalized
"""
function DataProjector(x, y, z, numDegrees;train=true, normalize=false)

    if !train
        #We flatten the data
        numSamples = length(x)*length(y)
        xData = repeat(x, inner=length(y))  # Repeat xData 
        yData = repeat(y, outer=length(x))  # Repeat yData 
        zData = vec(z) #Flattened Z data
    else
        numSamples = length(z)
        xData=x
        yData=y
        zData=z
    end
     #NxN
    numFeatures = (numDegrees + 1) * (numDegrees + 2) ÷ 2  # Number of polynomial terms
    DesignMatrix = zeros(numSamples, numFeatures)
    column=1
    for i in 0:numDegrees
        for j in 0:(numDegrees-i)
            DesignMatrix[:,column] .= (xData .^ i) .* (yData .^ j)
            column+=1
        end
    end
    if normalize
        col_std = zeros(column-1)
        col_mean = zeros(column-1)
        for j in 1:(column-1)
            col_mean[j] = mean(DesignMatrix[:, j])
            col_std[j] = std(DesignMatrix[:, j])

            # Avoid dividing by zero if the standard deviation is zero (constant column)
            if col_std[j] != 0
                DesignMatrix[:, j] = (DesignMatrix[:, j] .- col_mean[j]) ./ col_std[j]
            else
                DesignMatrix[:, j] = DesignMatrix[:, j]  # No standardization if std is 0
            end
        end
    end
    if train
        if !normalize
            return DesignMatrix
        else
            return DesignMatrix, col_std, col_mean
        end
    else
        if !normalize
            return DesignMatrix, zData
        else
            return DesignMatrix, zData, col_std, col_mean
        end
    end
end

function LassoReg(DesignMatrix, zData, γ)
    (T, K) = (size(DesignMatrix, 1), size(DesignMatrix, 2))
    Q = DesignMatrix'DesignMatrix #/ T
    c = DesignMatrix'zData #/ T                      #c'b = Y'X*b

    b = Variable(K)              #define variables to optimize over
    L1 = quadform(b, Q)            #b'Q*b
    L2 = dot(c, b)                 #c'b
    L3 = norm(b, 1)                #sum(|b|)

    # u'u/T + γ*sum(|b|) where u = Y-Xb
    problem = minimize(L1 - 2 * L2 + γ * L3)

    solve!(problem, SCS.Optimizer; silent = true)
    problem.status == Convex.MOI.OPTIMAL ? beta = vec(Convex.evaluate(b)) : beta = NaN

    return beta
end
"""
    Regression(DesignMatrix, zData, Method; λ)

Preforms a Regression with a DesignMatrix, zData and a given method


# Input
 - DesignMatrix: The designmatrix for the regression
 - zData:        The z data for the regression
 - Method:       The method, either; OLS, Ridge or Lasso
 - λ:            Parameter for Ridge and Lasso regression
"""
function Regression(DesignMatrix, zData, Method; λ=0)
    if Method=="OLS"
        Hessian = Transpose(DesignMatrix)*DesignMatrix
        beta = inv(Hessian)*Transpose(DesignMatrix)*zData
        
    elseif Method=="Ridge"
        Hessian = Transpose(DesignMatrix)*DesignMatrix
        beta = inv(Hessian+λ*I)*Transpose(DesignMatrix)*zData
        
    elseif Method=="Lasso"
        beta = LassoReg(DesignMatrix, zData, λ)
        
    else
        @warn "Method Undefined" 
        println("Expected: OLS, Lasso or Ridge, but got: ", Method)
        return
    end
    return beta
end

function MSE(DesignMatrix, z, beta; λ=0, γ=0)
    N = length(z)
    MSE = 1/N*sum((z .- DesignMatrix*beta).^2)+λ*(Transpose(beta)*beta) + γ*sum(abs.(beta))
    return MSE
end

function R2(DesignMatrix, zData, beta)
    N = length(zData)
    z_avg = 1/N*sum(zData)
    R2 = 1- sum((zData.-DesignMatrix*beta).^2)/sum((zData .- z_avg).^2)
    return R2
end